In [ ]:
import osmnx as ox

location_point = (39.1014537,-84.5124602)

# create network from point, inside bounding box of N, S, E, W each 750m from point
G2 = ox.graph_from_point(location_point, dist=19000, dist_type="bbox", network_type="drive")

In [ ]:
from keplergl import KeplerGl
import geopandas as gpd
# Create a KeplerGL map instance
map_1 = KeplerGl()


# Convert OSMnx graph data to a GeoDataFrame
nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(G2, nodes=True, edges=True)
# Add the OSMnx nodes and edges to the map as layers
map_1.add_data(data=nodes_osmnx, name="OSMnx Nodes")
map_1.add_data(data=edges_osmnx, name="OSMnx Edges")


# # Add the OSMnx street network to the map as a layer
# ox_layer = ox.plot_graph_folium(G, graph_map=map_1)

# # Add the Peartree graph network to the map as a layer
# pt_layer = pt.plot_graph(graph, graph_map=map_1)

# Display the map with both layers
map_1


In [ ]:
import geopandas as gpd
import osmnx as ox
import networkx as nx
# Path to the shapefile
# shapefile_path = "Cincinnati_Corporation_Line.shp"
shapefile_path = "Updated_shape.shx"

# Load the shapefile using geopandas
gdf = gpd.read_file(shapefile_path)

# Extract the geometry from the GeoDataFrame
polygon = gdf.geometry.iloc[50]
# print(len(gdf.geometry))
# print(type(polygon))
# Enable console logging
ox.config(log_console=True)

# Tags for filtering features
tags = {"highway":True}


# G1 = ox.graph_from_polygon(polygon,network_type="all")

# Extract features from the polygon using osmnx
# G1 = ox.features.features_from_polygon(polygon, tags)
# Create an empty MultiDiGraph to hold the combined results
G_combined = nx.MultiDiGraph()
crs = gdf.crs

# Set the CRS for the graph
G_combined.graph["crs"] = crs

dont_consider = [50,68,69]
# Iterate through each polygon and extract features
for index, polygon in gdf.iterrows():
    # Extract features from the current polygon using OSMnx
    print(index)
    if index not in dont_consider:
        print("here")
        G = ox.graph_from_polygon(polygon.geometry, network_type="walk")

        # Add the nodes and edges from the extracted graph to the combined graph
        G_combined.add_nodes_from(G.nodes(data=True))
        G_combined.add_edges_from(G.edges(data=True))

# At this point, G_combined contains all nodes and edges from all polygons
print("Number of nodes in the combined graph:", G_combined.number_of_nodes())
print("Number of edges in the combined graph:", G_combined.number_of_edges())

In [ ]:
from keplergl import KeplerGl

# Create a KeplerGl map instance
map_1 = KeplerGl()

# Convert OSMnx graph data to a GeoDataFrame
nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(G_combined, nodes=True, edges=True)
# Add the OSMnx nodes and edges to the map as layers
map_1.add_data(data=nodes_osmnx, name="OSMnx Nodes")
map_1.add_data(data=edges_osmnx, name="OSMnx Edges")


# Display the map
map_1

In [ ]:
import geopandas as gpd
import osmnx as ox
import networkx as nx
# Path to the shapefile
# shapefile_path = "Cincinnati_Corporation_Line.shp"
shapefile_path = "Updated_shape.shx"

# Load the shapefile using geopandas
gdf = gpd.read_file(shapefile_path)

# Extract the geometry from the GeoDataFrame
polygon = gdf.geometry.iloc[50]
# print(len(gdf.geometry))
# print(type(polygon))
# Enable console logging
ox.config(log_console=True)

# Tags for filtering features
tags = {"highway":True}


# G1 = ox.graph_from_polygon(polygon,network_type="all")

# Extract features from the polygon using osmnx
# G1 = ox.features.features_from_polygon(polygon, tags)
# Create an empty MultiDiGraph to hold the combined results
G_Combined_bike = nx.MultiDiGraph()
crs = gdf.crs

# Set the CRS for the graph
G_Combined_bike.graph["crs"] = crs

dont_consider = [50,68,69,70]
# Iterate through each polygon and extract features
for index, polygon in gdf.iterrows():
    # Extract features from the current polygon using OSMnx
    print(index)
    if index not in dont_consider:
        print("here")
        G = ox.graph_from_polygon(polygon.geometry, network_type="bike")

        # Add the nodes and edges from the extracted graph to the combined graph
        G_Combined_bike.add_nodes_from(G.nodes(data=True))
        G_Combined_bike.add_edges_from(G.edges(data=True))

# At this point, G_Combined_bike contains all nodes and edges from all polygons
print("Number of nodes in the combined graph:", G_Combined_bike.number_of_nodes())
print("Number of edges in the combined graph:", G_Combined_bike.number_of_edges())

In [ ]:
from keplergl import KeplerGl

# Create a KeplerGl map instance
map_1 = KeplerGl()

# Convert OSMnx graph data to a GeoDataFrame
nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(G_Combined_bike, nodes=True, edges=True)
# Add the OSMnx nodes and edges to the map as layers
map_1.add_data(data=nodes_osmnx, name="OSMnx Nodes")
map_1.add_data(data=edges_osmnx, name="OSMnx Edges")


# Display the map
map_1

In [ ]:
# for node, data in G_combined.nodes(data=True):
#     print("Node:", node)
#     print("Attributes:", data)

for u, v, key, data in G_Combined_bike.edges(data=True, keys=True):
    print("Edge:", u, v, key)
    print("Attributes:", data)

In [ ]:
import json

import pandas as pd
from geopandas import GeoDataFrame
from shapely.geometry import Point,LineString
# Load the JSON data
with open("gtfs_data.json", "r") as json_file:
    gtfs_data = json.load(json_file)


# Add GTFS nodes and edges from JSON data as a new layer in KeplerGl
gtfs_nodes = []
gtfs_edges = []
for node_entry in gtfs_data["nodes"]:
    node_id = node_entry["id"]
    node_attrs = node_entry["attributes"]
    gtfs_nodes.append({"id": node_id, **node_attrs})

for edge_entry in gtfs_data["edges"]:
    u = edge_entry["u"]
    v = edge_entry["v"]
    edge_attrs = edge_entry["attributes"]
    gtfs_edges.append({"u": u, "v": v, **edge_attrs})

    # Convert GTFS nodes to GeoDataFrame with synthetic geometry
# gtfs_nodes = gtfs_data["nodes"]

# Convert GTFS nodes to GeoDataFrame with synthetic geometry
gtfs_nodes_df = pd.DataFrame(gtfs_nodes)
print(gtfs_nodes_df)
gtfs_nodes_df["geometry"] = [Point(node["x"], node["y"]) for _, node in gtfs_nodes_df.iterrows()]
gtfs_nodes_gdf = GeoDataFrame(gtfs_nodes_df, geometry="geometry")

# Create a dictionary to map node IDs to their corresponding x, y coordinates
node_id_to_coords = {row['id']: (row['x'], row['y']) for _, row in gtfs_nodes_df.iterrows()}

# Convert GTFS edges to GeoDataFrame with proper geometry
gtfs_edges_df = pd.DataFrame(gtfs_edges)
gtfs_edges_df["geometry"] = [
    LineString([
        node_id_to_coords[node["u"]],
        node_id_to_coords[node["v"]]
    ])
    for _, node in gtfs_edges_df.iterrows()
]
gtfs_edges_gdf = GeoDataFrame(gtfs_edges_df, geometry="geometry")

In [ ]:
from keplergl import KeplerGl

# Create a KeplerGl map instance
map_2 = KeplerGl()

# Convert OSMnx graph data to a GeoDataFrame
nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(G_combined, nodes=True, edges=True)
# Add the OSMnx nodes and edges to the map as layers
map_2.add_data(data=nodes_osmnx, name="OSMnx Nodes")
map_2.add_data(data=edges_osmnx, name="OSMnx Edges")

map_2.add_data(data=gtfs_nodes_gdf, name="GTFS Nodes")
map_2.add_data(data=gtfs_edges_gdf, name="GTFS Edges")


# Display the map
map_2

In [ ]:
from keplergl import KeplerGl
import geopandas as gpd
# Create a KeplerGL map instance
map_1 = KeplerGl()


# Convert OSMnx graph data to a GeoDataFrame
# nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(G_interest, nodes=True, edges=True)
# # Add the OSMnx nodes and edges to the map as layers
# map_1.add_data(data=nodes_osmnx, name="OSMnx Nodes")
# map_1.add_data(data=edges_osmnx, name="OSMnx Edges")

for idx, graph in enumerate(Graph_array[0:10]):
    print(idx)
    node_name="OSMnx" + str(idx) + "Nodes"
    edge_name="OSMnx" + str(idx) + "Edges"

     # Convert node data to a DataFrame and add a geometry column
    nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(graph, nodes=True, edges=True)
    # Add the DataFrames as layers to the KeplerGl map
    map_1.add_data(data=nodes_osmnx, name=node_name)
    map_1.add_data(data=edges_osmnx, name=edge_name)


# Display the map with both layers
map_1

In [ ]:
from keplergl import KeplerGl
import geopandas as gpd
# Create a KeplerGL map instance
map_1 = KeplerGl()


# Convert OSMnx graph data to a GeoDataFrame
# nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(G_interest, nodes=True, edges=True)
# # Add the OSMnx nodes and edges to the map as layers
# map_1.add_data(data=nodes_osmnx, name="OSMnx Nodes")
# map_1.add_data(data=edges_osmnx, name="OSMnx Edges")

for idx, graph in enumerate(Graph_array[10:20]):
    print(idx)
    node_name="OSMnx" + str(idx) + "Nodes"
    edge_name="OSMnx" + str(idx) + "Edges"

     # Convert node data to a DataFrame and add a geometry column
    nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(graph, nodes=True, edges=True)
    # Add the DataFrames as layers to the KeplerGl map
    map_1.add_data(data=nodes_osmnx, name=node_name)
    map_1.add_data(data=edges_osmnx, name=edge_name)


# Display the map with both layers
map_1

In [ ]:
from keplergl import KeplerGl
import geopandas as gpd
# Create a KeplerGL map instance
map_1 = KeplerGl()


# Convert OSMnx graph data to a GeoDataFrame
# nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(G_interest, nodes=True, edges=True)
# # Add the OSMnx nodes and edges to the map as layers
# map_1.add_data(data=nodes_osmnx, name="OSMnx Nodes")
# map_1.add_data(data=edges_osmnx, name="OSMnx Edges")

for idx, graph in enumerate(Graph_array[20:30]):
    print(idx)
    node_name="OSMnx" + str(idx) + "Nodes"
    edge_name="OSMnx" + str(idx) + "Edges"

     # Convert node data to a DataFrame and add a geometry column
    nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(graph, nodes=True, edges=True)
    # Add the DataFrames as layers to the KeplerGl map
    map_1.add_data(data=nodes_osmnx, name=node_name)
    map_1.add_data(data=edges_osmnx, name=edge_name)


# Display the map with both layers
map_1

In [ ]:
from keplergl import KeplerGl
import geopandas as gpd
# Create a KeplerGL map instance
map_1 = KeplerGl()


# Convert OSMnx graph data to a GeoDataFrame
# nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(G_interest, nodes=True, edges=True)
# # Add the OSMnx nodes and edges to the map as layers
# map_1.add_data(data=nodes_osmnx, name="OSMnx Nodes")
# map_1.add_data(data=edges_osmnx, name="OSMnx Edges")

for idx, graph in enumerate(Graph_array[30:40]):
    print(idx)
    node_name="OSMnx" + str(idx) + "Nodes"
    edge_name="OSMnx" + str(idx) + "Edges"

     # Convert node data to a DataFrame and add a geometry column
    nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(graph, nodes=True, edges=True)
    # Add the DataFrames as layers to the KeplerGl map
    map_1.add_data(data=nodes_osmnx, name=node_name)
    map_1.add_data(data=edges_osmnx, name=edge_name)


# Display the map with both layers
map_1

In [ ]:
from keplergl import KeplerGl
import geopandas as gpd
# Create a KeplerGL map instance
map_1 = KeplerGl()


# Convert OSMnx graph data to a GeoDataFrame
# nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(G_interest, nodes=True, edges=True)
# # Add the OSMnx nodes and edges to the map as layers
# map_1.add_data(data=nodes_osmnx, name="OSMnx Nodes")
# map_1.add_data(data=edges_osmnx, name="OSMnx Edges")

for idx, graph in enumerate(Graph_array[40:50]):
    print(idx)
    node_name="OSMnx" + str(idx) + "Nodes"
    edge_name="OSMnx" + str(idx) + "Edges"

     # Convert node data to a DataFrame and add a geometry column
    nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(graph, nodes=True, edges=True)
    # Add the DataFrames as layers to the KeplerGl map
    map_1.add_data(data=nodes_osmnx, name=node_name)
    map_1.add_data(data=edges_osmnx, name=edge_name)


# Display the map with both layers
map_1

In [ ]:
from keplergl import KeplerGl
import geopandas as gpd
# Create a KeplerGL map instance
map_1 = KeplerGl()


# Convert OSMnx graph data to a GeoDataFrame
# nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(G_interest, nodes=True, edges=True)
# # Add the OSMnx nodes and edges to the map as layers
# map_1.add_data(data=nodes_osmnx, name="OSMnx Nodes")
# map_1.add_data(data=edges_osmnx, name="OSMnx Edges")

for idx, graph in enumerate(Graph_array[50:60]):
    print(idx)
    node_name="OSMnx" + str(idx) + "Nodes"
    edge_name="OSMnx" + str(idx) + "Edges"

     # Convert node data to a DataFrame and add a geometry column
    nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(graph, nodes=True, edges=True)
    # Add the DataFrames as layers to the KeplerGl map
    map_1.add_data(data=nodes_osmnx, name=node_name)
    map_1.add_data(data=edges_osmnx, name=edge_name)


# Display the map with both layers
map_1

In [ ]:
from keplergl import KeplerGl
import geopandas as gpd
# Create a KeplerGL map instance
map_1 = KeplerGl()


# Convert OSMnx graph data to a GeoDataFrame
# nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(G_interest, nodes=True, edges=True)
# # Add the OSMnx nodes and edges to the map as layers
# map_1.add_data(data=nodes_osmnx, name="OSMnx Nodes")
# map_1.add_data(data=edges_osmnx, name="OSMnx Edges")

for idx, graph in enumerate(Graph_array[60:70]):
    print(idx)
    node_name="OSMnx" + str(idx) + "Nodes"
    edge_name="OSMnx" + str(idx) + "Edges"

     # Convert node data to a DataFrame and add a geometry column
    nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(graph, nodes=True, edges=True)
    # Add the DataFrames as layers to the KeplerGl map
    map_1.add_data(data=nodes_osmnx, name=node_name)
    map_1.add_data(data=edges_osmnx, name=edge_name)


# Display the map with both layers
map_1

In [1]:
import geopandas as gpd
import osmnx as ox
import networkx as nx
# Path to the shapefile
# shapefile_path = "Cincinnati_Corporation_Line.shp"
shapefile_path = "Cleaned.shp"

# Load the shapefile using geopandas
gdf = gpd.read_file(shapefile_path)
polygon = gdf.geometry.iloc[0]
ox.config(log_console=True)

G_interest = ox.graph_from_polygon(polygon,network_type="walk")
print(G_interest)

/var/folders/fs/_tdftjp91mdchj_5_wfkr5g40000gn/T/ipykernel_18939/1885843762.py:11: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(log_console=True)


MultiDiGraph with 39725 nodes and 111902 edges


In [4]:
from keplergl import KeplerGl
import geopandas as gpd
# Create a KeplerGL map instance
map_1 = KeplerGl(height=600)


# Convert OSMnx graph data to a GeoDataFrame
nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(G_interest, nodes=True, edges=True)
# print(nodes_osmnx)
# Add the OSMnx nodes and edges to the map as layers
map_1.add_data(data=nodes_osmnx, name="OSMnx Nodes")
map_1.add_data(data=edges_osmnx, name="OSMnx Edges")

map_1.save_to_html(file_name='network.html')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Map saved to network.html!


In [2]:
import json
import pandas as pd
# Load the JSON data
with open("station_information.json", "r") as json_file:
    station_information= json.load(json_file)

station_info_df = pd.DataFrame(station_information['data']['stations'])
station_info_df.drop(columns=[ 'rental_uris', '_bcycle_station_type', 'region_id',
       'address'],inplace=True)
print(station_info_df)

         lon       lat                        name                station_id
0  -84.51164  39.10139             Fountain Square  bcycle_cincyredbike_2786
1  -84.51894  39.10435                   City Hall  bcycle_cincyredbike_2787
2  -84.51187  39.10385              Aronoff Center  bcycle_cincyredbike_2788
3  -84.50932  39.09849                   Ball Park  bcycle_cincyredbike_2789
4  -84.51056  39.09722              Freedom Center  bcycle_cincyredbike_2790
..       ...       ...                         ...                       ...
63 -84.51448  39.11447                  Grant Park  bcycle_cincyredbike_7332
64 -84.49741  39.11722         Gilbert & Eden Park  bcycle_cincyredbike_7569
65 -84.48969  39.12599             Peebles' Corner  bcycle_cincyredbike_7570
66 -84.49317  39.12599          McMillan & Concord  bcycle_cincyredbike_7572
67 -84.48465  39.12571  McMillan & Victory Parkway  bcycle_cincyredbike_7573

[68 rows x 4 columns]


In [ ]:
locations_df = station_info_df  # Replace with your file path

# Create an empty DataFrame to store the results
# results = pd.DataFrame(columns=['Location', 'Category', 'Amenity', 'Name', 'Distance'])
# Create an empty list to store the data for each row
data = []
name_exception=[]
categories = {
    "Food and Drink": ["restaurant", "cafe", "bar", "pub", "fast_food", "food_court", "ice_cream", "bakery"],
    "Shopping": ["supermarket", "convenience", "department_store", "clothes", "shoes", "electronics", "bookstore"],
    "Healthcare": ["hospital", "clinic", "pharmacy", "dentist", "doctors"],
    "Education": ["school", "college", "university", "library"],
    "Entertainment": ["cinema", "theatre", "nightclub", "casino", "arts_centre"],
    "Recreation": ["park", "gym", "sports_centre", "playground", "stadium", "swimming_pool"],
    "Transportation": ["bus_station", "train_station", "subway_station", "airport", "taxi"],
    "Lodging": ["hotel", "motel", "guest_house", "hostel"],
    "Finance": ["bank", "atm"],
    "Religion": ["place_of_worship", "church", "mosque", "synagogue", "temple"],
    "Services": ["post_office", "police", "fire_station", "townhall", "courthouse"],
    "Other": ["public_building", "office", "fuel", "car_rental", "car_wash"],
}

# Updated tags dictionary
tags = {
    'amenity': [
        'restaurant', 'cafe', 'bar', 'pub', 'fast_food', 'food_court', 'ice_cream', 'bakery',
        'hospital', 'clinic', 'pharmacy', 'dentist', 'doctors',
        'school', 'college', 'university', 'library',
        'cinema', 'theatre', 'nightclub', 'casino', 'arts_centre',
        'park', 'gym', 'sports_centre', 'playground', 'stadium', 'swimming_pool',
        'bus_station', 'train_station', 'subway_station', 'airport', 'taxi',
        'hotel', 'motel', 'guest_house', 'hostel',
        'bank', 'atm',
        'place_of_worship', 'church', 'mosque', 'synagogue', 'temple',
        'post_office', 'police', 'fire_station', 'townhall', 'courthouse',
        'public_building', 'office', 'fuel', 'car_rental', 'car_wash','bus_stop'
    ],
    'building': [
        'apartments', 'bungalow', 'cabin', 'detached', 'dormitory', 'farm', 'house', 'residential', 'semidetached_house',
        'terrace', 'houseboat', 'static_caravan', 'commercial', 'industrial', 'retail', 'warehouse', 'cathedral', 'chapel',
        'mosque', 'temple', 'synagogue', 'shrine', 'civic', 'hospital', 'school', 'stadium', 'train_station',
        'university', 'barn', 'bridge', 'bunker', 'cabin', 'construction', 'cowshed', 'farm', 'garage',
        'garages', 'greenhouse', 'hangar', 'hut', 'roof', 'shed', 'stable', 'sty', 'transformer_tower', 'service',
        'kiosk', 'ruins', 'train_station', 'service', 'university', 'wall'
    ],
    'tourism': [
        'alpine_hut', 'aquarium', 'artwork', 'attraction', 'bed_and_breakfast', 'cabin', 'camp_site',
        'caravan_site', 'chalet', 'gallery', 'guest_house', 'holiday_apartment', 'hotel', 'information', 'motel',
        'museum', 'picnic_site', 'theme_park', 'viewpoint', 'zoo'
    ],
    'leisure': [
        'fitness_centre', 'golf_course', 'ice_rink', 'miniature_golf', 'nature_reserve', 'park', 'pitch', 'playground',
        'sports_centre', 'stadium', 'swimming_pool', 'track', 'water_park', 'common', 'dance', 'dog_park', 'firepit',
        'fishing', 'fishing_area', 'garden', 'garden_centre', 'ice_rink', 'marina', 'sauna', 'slipway', 'sports_centre',
        'stadium', 'swimming_area', 'swimming_pool', 'track', 'water_park', 'wildlife_hide'
    ],
    'shop': [
        'alcohol', 'bakery', 'beverages', 'butcher', 'cheese', 'chocolate', 'coffee', 'confectionery', 'convenience',
        'deli', 'dairy', 'farm', 'frozen_food', 'greengrocer', 'healthfood', 'ice_cream', 'organic', 'pasta', 'pastry',
        'seafood', 'spices', 'tea', 'wine', 'department_store', 'mall', 'supermarket', 'bag', 'boutique', 'clothes',
        'fabric', 'fashion', 'jewelry', 'leather', 'shoes', 'tailor', 'watches', 'baby_goods', 'bag', 'bakery', 'beauty',
        'bed', 'boutique', 'butcher', 'car', 'car_parts', 'car_repair', 'carpet', 'charity', 'chemist', 'clothes',
        'computer', 'convenience', 'copyshop', 'cosmetics', 'craft', 'deli', 'department_store', 'discount', 'doityourself',
        'dry_cleaning', 'electronics', 'erotic', 'fabric', 'florist', 'funeral_directors', 'furniture', 'garden_centre',
        'gas', 'gift', 'greengrocer', 'grocery', 'hairdresser', 'hardware', 'hearing_aids', 'hifi', 'interior_decoration',
        'jewelry', 'kitchen', 'mall', 'mobile_phone', 'motorcycle', 'music', 'musical_instrument', 'newsagent', 'optician',
        'outdoor', 'perfumery', 'shoes', 'sports', 'tailor', 'ticket', 'toys', 'travel_agency', 'tyres', 'variety_store',
        'video', 'video_games', 'watches', 'weapons'
    ],
    'highway': ['bus_stop'],
    'boundary': ['administrative', 'maritime', 'national_park', 'protected_area', 'postal_code', 'political', 'civil'],
    'landuse': ['residential', 'commercial', 'industrial', 'retail', 'warehouse', 'construction', 'school', 'university', 'hospital',
    'public', 'religious', 'cemetery', 'military', 'farmland', 'farmyard', 'orchard', 'vineyard', 'plant_nursery',
    'allotments', 'meadow', 'grass', 'forest', 'wood', 'wetland', 'recreation_ground', 'park', 'playground', 'stadium',
    'sports_centre', 'swimming_pool', 'pitch', 'golf_course', 'raceway', 'track', 'quarry', 'basin', 'reservoir',
    'wastewater', 'basin', 'landfill', 'salt_pond', 'pond'],
}

# Loop through each location in the DataFrame
for index, row in locations_df.iterrows():
    print(index)
    lat, lon = row['lat'], row['lon']
    # Get the nearest node in the walking network to the location
    orig_node = ox.distance.nearest_nodes(G_interest, lon, lat)
    print(orig_node)
    # Calculate the walking network distances to amenities
    for category, types in tags.items():
        try:
            amenities = ox.features.features_from_point((lat, lon), dist=500, tags={category: types})
        except Exception as e:
            print(f"Error processing category: {category} for {index}. Skipping...\n{e}")
            continue
        for idx, amenity in amenities.iterrows():
            # Access the geometry of the amenity
            amenity_geom = amenity['geometry']
            if amenity_geom.geom_type == 'Polygon':
                amenity_point = amenity_geom.centroid
                amenity_x = amenity_point.x
                amenity_y = amenity_point.y
            elif amenity_geom.geom_type == 'MultiPolygon':
                amenity_centroid = amenity_geom.centroid
                amenity_x = amenity_centroid.x
                amenity_y = amenity_centroid.y
            elif amenity_geom.geom_type == 'LineString':
                amenity_centroid = amenity_geom.centroid
                amenity_x = amenity_centroid.x
                amenity_y = amenity_centroid.y
            else:
                amenity_x = amenity_geom.x
                amenity_y = amenity_geom.y
            
            # Calculate the network distance
            dest_node = ox.distance.nearest_nodes(G_interest, amenity_x, amenity_y)
            print(dest_node)
            try:
                distance = nx.shortest_path_length(G_interest, orig_node, dest_node, weight='length')
                if distance <= 500:  # Check if distance exceeds 500 meters

                    # Check if the name key exists and use it, otherwise use a default value
                    amenity_name = amenity.get("name", "Unknown")
                    if amenity_name == "Unknown":
                        name_exception.append(amenity)
                    # Append the data for the row to the list
                    data.append({
                        'Location': (lat, lon),
                        'Station ID':row['station_id'],
                        'Category': category,
                        'Amenity': amenity[category],
                        'Amenity Location': (amenity_y, amenity_x),
                        'Name': amenity_name,
                        'Distance': distance
                    })
            except nx.NetworkXNoPath:
                pass  # Handle cases where no path exists

# Create a DataFrame from the list of data
results = pd.DataFrame(data)

# Print the results
print(results)

In [ ]:
results.to_csv('amenity_distances2.csv', index=False)
print(name_exception)

In [31]:
from keplergl import KeplerGl
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point,LineString
# Create a KeplerGl map instance
map_1 = KeplerGl(height=800, 
    config={
    "version": "v1",
    "config": {
        "mapState": {
            "latitude": 39.10139,
            "longitude": -84.51164,
            "zoom": 13
        },
        # "visState": {
        #     "layers": [
        #         {
        #             "id": "unique_amenity_locations",
        #             "type": "geojson",
        #             "config": {
        #                 "dataId": "Unique Amenity Locations",
        #                 "label": "Amenity Locations",
        #                 "color": [255,203,153],
        #                 "highlightColor": [252,242,26,255],
        #                 "columns": {
        #                     "geojson": "geometry"
        #                 },
        #                 "isVisible": True,
        #                 "visConfig": {
        #                     "opacity": 0.8,
        #                     "strokeOpacity": 0.8,
        #                     "strokeColor": None,
        #                     "thickness": 0.5,
        #                     "colorRange": {
        #                         "name": "Uber Viz Qualitative 3",
        #                         "type": "qualitative",
        #                         "category": "Uber",
        #                         "colors": [
        #                             "#12939A",
        #                             "#DDB27C",
        #                             "#88572C",
        #                             "#FF991F",
        #                             "#F15C17",
        #                             "#223F9A",
        #                             "#DA70BF",
        #                             "#125C77",
        #                             "#4DC19C",
        #                             "#776E57",
        #                             "#17B8BE",
        #                             "#F6D18A",
        #                             "#B7885E",
        #                             "#FFCB99",
        #                             "#F89570"
        #                         ]
        #                     },
        #                     "strokeColorRange": {
        #                         "name": "Global Warming",
        #                         "type": "sequential",
        #                         "category": "Uber",
        #                         "colors": [
        #                         "#5A1846",
        #                         "#900C3F",
        #                         "#C70039",
        #                         "#E3611C",
        #                         "#F1920E",
        #                         "#FFC300"
        #                         ]
        #                     },
        #                     "radius": 10,
        #                     "sizeRange": [
        #                         0,
        #                         10
        #                     ],
        #                     "radiusRange": [
        #                         0,
        #                         50
        #                     ],
        #                     "heightRange": [
        #                         0,
        #                         500
        #                     ],
        #                     "elevationScale": 5,
        #                     "enableElevationZoomFactor": True,
        #                     "stroked": False,
        #                     "filled": True,
        #                     "enable3d": False,
        #                     "wireframe": False
        #                 }
        #             },
        #             "visualChannels": {
        #             "colorField": {
        #             "name": "Amenity",
        #             "type": "string"
        #             },
        #             "colorScale": "ordinal",
        #             "strokeColorField": None,
        #             "strokeColorScale": "quantile",
        #             "sizeField": None,
        #             "sizeScale": "linear",
        #             "heightField": None,
        #             "heightScale": "linear",
        #             "radiusField": None,
        #             "radiusScale": "linear"
        #             }
        #         }
        #     ]
        # }
    }
}
)

results_df=pd.read_csv("amenity_distances2.csv")
# print(results_df)
# Convert OSMnx graph data to a GeoDataFrame
nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(G_interest, nodes=True, edges=True)
# Add the OSMnx nodes and edges to the map as layers
map_1.add_data(data=nodes_osmnx, name="OSMnx Nodes")
map_1.add_data(data=edges_osmnx, name="OSMnx Edges")

# Step 3: Extract unique points from 'Location' and 'Amenity Location' columns
unique_locations = results_df['Location'].apply(eval).unique()
# Create a DataFrame for unique amenity locations including 'Name'
unique_amenity_locations = results_df[['Amenity Location', 'Name', 'Amenity']].apply(
    lambda row: (row['Amenity Location'], row['Name'], row['Amenity']), axis=1).apply(
    lambda loc: (eval(loc[0])[0], eval(loc[0])[1], loc[1], loc[2])).unique()

# Step 4: Create GeoDataFrames from the unique points
locations_gdf = gpd.GeoDataFrame(
    geometry=[Point(lon, lat) for lat, lon in unique_locations],
    crs="EPSG:4326"
)
print(unique_amenity_locations)
amenity_locations_gdf = gpd.GeoDataFrame(
    geometry=[Point(lon, lat) for lat, lon, name, amenity in unique_amenity_locations],
    crs="EPSG:4326"
)
# Add the 'Name' column to amenity_locations_gdf
amenity_locations_gdf['Name'] = [name for lat, lon, name, amenity in unique_amenity_locations]
amenity_locations_gdf['Amenity'] = [amenity for lat, lon, name, amenity in unique_amenity_locations]

map_1.add_data(data=locations_gdf, name="Unique Locations")
map_1.add_data(data=amenity_locations_gdf, name="Unique Amenity Locations")



# Group amenity locations by Amenity type and add them as separate layers
unique_amenities = amenity_locations_gdf['Amenity'].unique()
print(unique_amenities)
amenity_gdfs = {}
for amenity_type in unique_amenities:
    subset_gdf = amenity_locations_gdf[amenity_locations_gdf['Amenity'] == amenity_type]
    name=amenity_type+".csv"
    subset_gdf.to_csv(name)
#     amenity_gdfs[amenity_type] = subset_gdf
    # map_1.add_data(data=subset_gdf, name=f"Amenity Type: {amenity_type}")
# print(amenity_gdfs)
# Add separate amenity layers to the map
# for amenity_type, gdf in amenity_gdfs.items():
#     map_1.add_data(data=gdf, name=f"Amenity Type: {amenity_type}")

# Create lines between locations and their corresponding amenity locations
lines = []
for _, row in results_df.iterrows():
    location = Point(eval(row['Location'])[1], eval(row['Location'])[0])
    amenity_location = Point(eval(row['Amenity Location'])[1], eval(row['Amenity Location'])[0])
    line = LineString([location, amenity_location])
    lines.append({'geometry': line, 'Distance': row['Distance']})
# print(lines)
lines_gdf = gpd.GeoDataFrame(lines, crs="EPSG:4326")
map_1.add_data(data=lines_gdf, name="Lines")

# Display the map
map_1.save_to_html(file_name='amenities_with_network.html')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


[(39.1006371, -84.5109365, 'Mercantile Library', 'library')
 (39.1008837, -84.5127421, "McCormick & Schmick's", 'restaurant')
 (39.1014923, -84.5118525, "Graeter's", 'ice_cream') ...
 (39.127443460706345, -84.48472166179573, 'Top Up Mart', 'convenience')
 (39.1252029, -84.4814066, 'McMillan St & Bell Pl', 'bus_stop')
 (39.1253891, -84.4830933, 'McMillan St & Ashland Ave', 'bus_stop')]


/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


['library' 'restaurant' 'ice_cream' 'cafe' 'post_office' 'bank' 'bar'
 'fast_food' 'atm' 'bus_station' 'theatre' 'place_of_worship' 'courthouse'
 'apartments' 'cathedral' 'chapel' 'retail' 'construction' 'roof'
 'commercial' 'information' 'artwork' 'hotel' 'museum' 'gallery'
 'viewpoint' 'attraction' 'park' 'ice_rink' 'copyshop' 'erotic' 'mall'
 'alcohol' 'hifi' 'clothes' 'optician' 'convenience' 'jewelry' 'shoes'
 'sports' 'variety_store' 'ticket' 'bus_stop' 'car_rental' 'dentist'
 'townhall' 'school' 'residential' 'garage' 'playground' 'garden'
 'swimming_pool' 'pitch' 'pharmacy' 'pub' 'nightclub' 'fitness_centre'
 'furniture' 'doityourself' 'hardware' 'hairdresser' 'police' 'stadium'
 'common' 'mobile_phone' 'picnic_site' 'sports_centre' 'clinic' 'car_wash'
 'food_court' 'arts_centre' 'fire_station' 'casino' 'house' 'fabric'
 'bakery' 'gift' 'confectionery' 'beverages' 'car_repair' 'supermarket'
 'travel_agency' 'beauty' 'dog_park' 'cinema' 'music' 'florist' 'deli'
 'industrial' 'te

/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Map saved to amenities_with_network.html!


In [3]:

import osmnx as ox
import networkx as nx
import geopandas as gpd
import math

# Find the minimum and maximum latitudes and longitudes
min_lat = station_info_df['lat'].min()
max_lat = station_info_df['lat'].max()
min_lon = station_info_df['lon'].min()
max_lon = station_info_df['lon'].max()


# Calculate latitude and longitude differences for 1 kilometer extension
lat_diff = 5 / 111  # 1 kilometer is approximately 1/111 degrees of latitude
avg_lat = (min_lat + max_lat) / 2
lon_diff = 1 / (111.32 * math.cos(math.radians(avg_lat)))

# Extend the bounding box
min_lat -= lat_diff
max_lat += lat_diff
min_lon -= lon_diff
max_lon += lon_diff


north = max_lat
south = min_lat
east = max_lon
west = min_lon

tags={'amenity':True,'building':True,'historic':True,'leisure':True,'landuse':True,'tourism':True,'office':True,'shop':True}

# print(north,south,east,west)
# print(north+.005,south+.005,east-.005,west-.005)
G_bbox=ox.graph_from_bbox(north=north,south=south,east=east,west=west,network_type="walk",simplify=True)
print(G_bbox)


MultiDiGraph with 58885 nodes and 162848 edges


In [4]:

import osmnx as ox
import networkx as nx
import geopandas as gpd
import math

# Find the minimum and maximum latitudes and longitudes
min_lat = station_info_df['lat'].min()
max_lat = station_info_df['lat'].max()
min_lon = station_info_df['lon'].min()
max_lon = station_info_df['lon'].max()


# Calculate latitude and longitude differences for 1 kilometer extension
lat_diff = 5 / 111  # 1 kilometer is approximately 1/111 degrees of latitude
avg_lat = (min_lat + max_lat) / 2
lon_diff = 1 / (111.32 * math.cos(math.radians(avg_lat)))

# Extend the bounding box
min_lat -= lat_diff
max_lat += lat_diff
min_lon -= lon_diff
max_lon += lon_diff


north = max_lat
south = min_lat
east = max_lon
west = min_lon

tags={'amenity':True,'building':True,'historic':True,'leisure':True,'landuse':True,'tourism':True,'office':True,'shop':True}

# print(north,south,east,west)
# print(north+.005,south+.005,east-.005,west-.005)
G_bbox_bike=ox.graph_from_bbox(north=north,south=south,east=east,west=west,network_type="bike",simplify=True)
print(G_bbox_bike)


MultiDiGraph with 41207 nodes and 98142 edges


In [5]:
from keplergl import KeplerGl
import geopandas as gpd
# Create a KeplerGL map instance
map_1 = KeplerGl(height=600)


# Convert OSMnx graph data to a GeoDataFrame
nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(G_bbox_bike, nodes=True, edges=True)
# print(nodes_osmnx)
# Add the OSMnx nodes and edges to the map as layers
map_1.add_data(data=nodes_osmnx, name="OSMnx Nodes")
map_1.add_data(data=edges_osmnx, name="OSMnx Edges")

map_1.save_to_html(file_name='network_bike.html')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Map saved to network_bike.html!


In [4]:
locations_df = station_info_df  # Replace with your file path

# Create an empty DataFrame to store the results
# results = pd.DataFrame(columns=['Location', 'Category', 'Amenity', 'Name', 'Distance'])
# Create an empty list to store the data for each row
data = []
name_exception=[]
categories = {
    "Food and Drink": [
        'restaurant', 'cafe', 'bar', 'pub', 'fast_food', 'food_court', 'ice_cream', 'bakery',
    ],
    "Healthcare": [
        'hospital', 'clinic', 'pharmacy', 'dentist', 'doctors',
    ],
    "Education": [
        'school', 'college', 'university', 'library',
    ],
    "Entertainment": [
        'cinema', 'theatre', 'nightclub', 'casino', 'arts_centre',
    ],
    "Recreation": [
        'park', 'gym', 'sports_centre', 'playground', 'stadium', 'swimming_pool',
    ],
    "Transportation": [
        'bus_station', 'train_station', 'subway_station', 'airport', 'taxi',
    ],
    "Lodging": [
        'hotel', 'motel', 'guest_house', 'hostel',
    ],
    "Finance": [
        'bank', 'atm',
    ],
    "Religion": [
        'place_of_worship', 'church', 'mosque', 'synagogue', 'temple',
    ],
    "Services": [
        'post_office', 'police', 'fire_station', 'townhall', 'courthouse',
    ],
    "Other": [
        'public_building', 'office', 'fuel', 'car_rental', 'car_wash',
    ],
    "Building": [
        'apartments', 'bungalow', 'cabin', 'detached', 'dormitory', 'farm', 'house', 'residential',
        'semidetached_house', 'terrace', 'houseboat', 'static_caravan', 'commercial', 'industrial',
        'retail', 'warehouse', 'cathedral', 'chapel', 'mosque', 'temple', 'synagogue', 'shrine',
        'civic', 'hospital', 'school', 'stadium', 'train_station', 'university', 'barn', 'bridge',
        'bunker', 'cabin', 'construction', 'cowshed', 'farm', 'garage', 'garages', 'greenhouse',
        'hangar', 'hut', 'roof', 'shed', 'stable', 'sty', 'transformer_tower', 'service', 'kiosk',
        'ruins', 'train_station', 'service', 'university', 'wall',
    ],
    "Tourism": [
        'alpine_hut', 'aquarium', 'artwork', 'attraction', 'bed_and_breakfast', 'cabin', 'camp_site',
        'caravan_site', 'chalet', 'gallery', 'guest_house', 'holiday_apartment', 'hotel', 'information',
        'motel', 'museum', 'picnic_site', 'theme_park', 'viewpoint', 'zoo',
    ],
    "Leisure": [
        'fitness_centre', 'golf_course', 'ice_rink', 'miniature_golf', 'nature_reserve', 'park',
        'pitch', 'playground', 'sports_centre', 'stadium', 'swimming_pool', 'track', 'water_park',
        'common', 'dance', 'dog_park', 'firepit', 'fishing', 'fishing_area', 'garden', 'garden_centre',
        'ice_rink', 'marina', 'sauna', 'slipway', 'sports_centre', 'stadium', 'swimming_area',
        'swimming_pool', 'track', 'water_park', 'wildlife_hide',
    ],
    "Shop": [
        'alcohol', 'bakery', 'beverages', 'butcher', 'cheese', 'chocolate', 'coffee', 'confectionery',
        'convenience', 'deli', 'dairy', 'farm', 'frozen_food', 'greengrocer', 'healthfood', 'ice_cream',
        'organic', 'pasta', 'pastry', 'seafood', 'spices', 'tea', 'wine', 'department_store', 'mall',
        'supermarket', 'bag', 'boutique', 'clothes', 'fabric', 'fashion', 'jewelry', 'leather', 'shoes',
        'tailor', 'watches', 'baby_goods', 'bag', 'bakery', 'beauty', 'bed', 'boutique', 'butcher', 'car',
        'car_parts', 'car_repair', 'carpet', 'charity', 'chemist', 'clothes', 'computer', 'convenience',
        'copyshop', 'cosmetics', 'craft', 'deli', 'department_store', 'discount', 'doityourself', 'dry_cleaning',
        'electronics', 'erotic', 'fabric', 'florist', 'funeral_directors', 'furniture', 'garden_centre',
        'gas', 'gift', 'greengrocer', 'grocery', 'hairdresser', 'hardware', 'hearing_aids', 'hifi',
        'interior_decoration', 'jewelry', 'kitchen', 'mall', 'mobile_phone', 'motorcycle', 'music',
        'musical_instrument', 'newsagent', 'optician', 'outdoor', 'perfumery', 'shoes', 'sports', 'tailor',
        'ticket', 'toys', 'travel_agency', 'tyres', 'variety_store', 'video', 'video_games', 'watches', 'weapons',
    ]
}

# Updated tags dictionary
tags = {
    'amenity': [
        'restaurant', 'cafe', 'bar', 'pub', 'fast_food', 'food_court', 'ice_cream', 'bakery',
        'hospital', 'clinic', 'pharmacy', 'dentist', 'doctors',
        'school', 'college', 'university', 'library',
        'cinema', 'theatre', 'nightclub', 'casino', 'arts_centre',
        'park', 'gym', 'sports_centre', 'playground', 'stadium', 'swimming_pool',
        'bus_station', 'train_station', 'subway_station', 'airport', 'taxi',
        'hotel', 'motel', 'guest_house', 'hostel',
        'bank', 'atm',
        'place_of_worship', 'church', 'mosque', 'synagogue', 'temple',
        'post_office', 'police', 'fire_station', 'townhall', 'courthouse',
        'public_building', 'office', 'fuel', 'car_rental', 'car_wash','bus_stop'
    ],
    'building': [
        'apartments', 'bungalow', 'cabin', 'detached', 'dormitory', 'farm', 'house', 'residential', 'semidetached_house',
        'terrace', 'houseboat', 'static_caravan', 'commercial', 'industrial', 'retail', 'warehouse', 'cathedral', 'chapel',
        'mosque', 'temple', 'synagogue', 'shrine', 'civic', 'hospital', 'school', 'stadium', 'train_station',
        'university', 'barn', 'bridge', 'bunker', 'cabin', 'construction', 'cowshed', 'farm', 'garage',
        'garages', 'greenhouse', 'hangar', 'hut', 'roof', 'shed', 'stable', 'sty', 'transformer_tower', 'service',
        'kiosk', 'ruins', 'train_station', 'service', 'university', 'wall'
    ],
    'tourism': [
        'alpine_hut', 'aquarium', 'artwork', 'attraction', 'bed_and_breakfast', 'cabin', 'camp_site',
        'caravan_site', 'chalet', 'gallery', 'guest_house', 'holiday_apartment', 'hotel', 'information', 'motel',
        'museum', 'picnic_site', 'theme_park', 'viewpoint', 'zoo'
    ],
    'leisure': [
        'fitness_centre', 'golf_course', 'ice_rink', 'miniature_golf', 'nature_reserve', 'park', 'pitch', 'playground',
        'sports_centre', 'stadium', 'swimming_pool', 'track', 'water_park', 'common', 'dance', 'dog_park', 'firepit',
        'fishing', 'fishing_area', 'garden', 'garden_centre', 'ice_rink', 'marina', 'sauna', 'slipway', 'sports_centre',
        'stadium', 'swimming_area', 'swimming_pool', 'track', 'water_park', 'wildlife_hide'
    ],
    'shop': [
        'alcohol', 'bakery', 'beverages', 'butcher', 'cheese', 'chocolate', 'coffee', 'confectionery', 'convenience',
        'deli', 'dairy', 'farm', 'frozen_food', 'greengrocer', 'healthfood', 'ice_cream', 'organic', 'pasta', 'pastry',
        'seafood', 'spices', 'tea', 'wine', 'department_store', 'mall', 'supermarket', 'bag', 'boutique', 'clothes',
        'fabric', 'fashion', 'jewelry', 'leather', 'shoes', 'tailor', 'watches', 'baby_goods', 'bag', 'bakery', 'beauty',
        'bed', 'boutique', 'butcher', 'car', 'car_parts', 'car_repair', 'carpet', 'charity', 'chemist', 'clothes',
        'computer', 'convenience', 'copyshop', 'cosmetics', 'craft', 'deli', 'department_store', 'discount', 'doityourself',
        'dry_cleaning', 'electronics', 'erotic', 'fabric', 'florist', 'funeral_directors', 'furniture', 'garden_centre',
        'gas', 'gift', 'greengrocer', 'grocery', 'hairdresser', 'hardware', 'hearing_aids', 'hifi', 'interior_decoration',
        'jewelry', 'kitchen', 'mall', 'mobile_phone', 'motorcycle', 'music', 'musical_instrument', 'newsagent', 'optician',
        'outdoor', 'perfumery', 'shoes', 'sports', 'tailor', 'ticket', 'toys', 'travel_agency', 'tyres', 'variety_store',
        'video', 'video_games', 'watches', 'weapons'
    ]
}

# Loop through each location in the DataFrame
for index, row in locations_df.iterrows():
    print(index)
    lat, lon = row['lat'], row['lon']
    # Get the nearest node in the walking network to the location
    orig_node = ox.distance.nearest_nodes(G_bbox, lon, lat)
    print(orig_node)
    # Calculate the walking network distances to amenities
    for category, types in tags.items():
        try:
            amenities = ox.features.features_from_point((lat, lon), dist=500, tags={category: types})
        except Exception as e:
            print(f"Error processing category: {category} for {index}. Skipping...\n{e}")
            continue
        for idx, amenity in amenities.iterrows():
            # Access the geometry of the amenity
            amenity_geom = amenity['geometry']
            if amenity_geom.geom_type == 'Polygon':
                amenity_point = amenity_geom.centroid
                amenity_x = amenity_point.x
                amenity_y = amenity_point.y
            elif amenity_geom.geom_type == 'MultiPolygon':
                amenity_centroid = amenity_geom.centroid
                amenity_x = amenity_centroid.x
                amenity_y = amenity_centroid.y
            elif amenity_geom.geom_type == 'LineString':
                amenity_centroid = amenity_geom.centroid
                amenity_x = amenity_centroid.x
                amenity_y = amenity_centroid.y
            else:
                amenity_x = amenity_geom.x
                amenity_y = amenity_geom.y
            
            # Calculate the network distance
            dest_node = ox.distance.nearest_nodes(G_bbox, amenity_x, amenity_y)
            print(dest_node)
            try:
                distance= nx.shortest_path_length(G_bbox, orig_node, dest_node, weight='length')
                if distance <= 500:  # Check if distance exceeds 500 meters
                    
                    amenity_name = amenity.get("name", "Unknown")
                    if amenity_name == "Unknown":
                        name_exception.append(amenity)
                    # Append the data for the row to the list
                    data.append({
                        'Location': (lat, lon),
                        'Station ID':row['station_id'],
                        'Category': category,
                        'Amenity': amenity[category],
                        'Amenity Location': (amenity_y, amenity_x),
                        'Name': amenity_name,
                        'Distance': distance,
                    })
            except nx.NetworkXNoPath:
                pass  # Handle cases where no path exists

# Create a DataFrame from the list of data
results = pd.DataFrame(data)

# Print the results
print(results)

0
5471876971
6316883736
197531272
5471876965
6821484368
5471876971
5472286153
9765761444
1060225073
197485154
2471276565
5464268376
6833013001
6833013000
6833013000
5472286169
5471780396
1060108729
5471876948
9765830394
6821488805
197473167
6316880728
2046030667
6630316238
6833083832
197522439
7610874721
197616190
2471274155
197531256
2774846077
1866317811
5471876852
197436234
477195066
6630312444
197528400
6770749508
197467520
197531262
6745824487
197531262
197621307
197489284
197479039
5471780417
197514760
197467520
197467520
197436225
5464268587
6630316240
6621605233
6833012891
8962552676
197555875
197485154
197467526
197489290
197531265
197531272
9765864956
6744909067
197467526
2563714227
2345137217
197613946
197558196
6316883737
5471876975
6621605233
6833012891
197621307
197531256
197531256
197531256
197531256
197531256
5471876843
5471876970
6316883736
197485144
5471876971
5471876985
5471876985
6621605233
197473175
7071346298
197485154
1060084105
197436224
197436234
2774846091
204

In [5]:
results.to_csv('amenity_distances3.csv', index=False)
print(name_exception)

[shop                           convenience
geometry    POINT (-84.5007208 39.1347776)
Name: (node, 7649865426), dtype: object, artwork_type                               statue
species                     Mammuthus primigenius
species:wikidata                         Q3699044
subject                            woolly mammoth
subject:wikidata                         Q3699044
tourism                                   artwork
geometry            POINT (-84.5262337 39.099562)
nodes                                         NaN
website                                       NaN
Name: (node, 6668786081), dtype: object, artwork_type                                statue
species                      Mammuthus primigenius
species:wikidata                          Q3699044
subject                             woolly mammoth
subject:wikidata                          Q3699044
tourism                                    artwork
geometry            POINT (-84.5262506 39.0995424)
nodes                   

In [6]:
from keplergl import KeplerGl
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point,LineString
# Create a KeplerGl map instance
map_1 = KeplerGl(height=800, 
    config={
    "version": "v1",
    "config": {
        "mapState": {
            "latitude": 39.10139,
            "longitude": -84.51164,
            "zoom": 13
        },
        # "visState": {
        #     "layers": [
        #         {
        #             "id": "unique_amenity_locations",
        #             "type": "geojson",
        #             "config": {
        #                 "dataId": "Unique Amenity Locations",
        #                 "label": "Amenity Locations",
        #                 "color": [255,203,153],
        #                 "highlightColor": [252,242,26,255],
        #                 "columns": {
        #                     "geojson": "geometry"
        #                 },
        #                 "isVisible": True,
        #                 "visConfig": {
        #                     "opacity": 0.8,
        #                     "strokeOpacity": 0.8,
        #                     "strokeColor": None,
        #                     "thickness": 0.5,
        #                     "colorRange": {
        #                         "name": "Uber Viz Qualitative 3",
        #                         "type": "qualitative",
        #                         "category": "Uber",
        #                         "colors": [
        #                             "#12939A",
        #                             "#DDB27C",
        #                             "#88572C",
        #                             "#FF991F",
        #                             "#F15C17",
        #                             "#223F9A",
        #                             "#DA70BF",
        #                             "#125C77",
        #                             "#4DC19C",
        #                             "#776E57",
        #                             "#17B8BE",
        #                             "#F6D18A",
        #                             "#B7885E",
        #                             "#FFCB99",
        #                             "#F89570"
        #                         ]
        #                     },
        #                     "strokeColorRange": {
        #                         "name": "Global Warming",
        #                         "type": "sequential",
        #                         "category": "Uber",
        #                         "colors": [
        #                         "#5A1846",
        #                         "#900C3F",
        #                         "#C70039",
        #                         "#E3611C",
        #                         "#F1920E",
        #                         "#FFC300"
        #                         ]
        #                     },
        #                     "radius": 10,
        #                     "sizeRange": [
        #                         0,
        #                         10
        #                     ],
        #                     "radiusRange": [
        #                         0,
        #                         50
        #                     ],
        #                     "heightRange": [
        #                         0,
        #                         500
        #                     ],
        #                     "elevationScale": 5,
        #                     "enableElevationZoomFactor": True,
        #                     "stroked": False,
        #                     "filled": True,
        #                     "enable3d": False,
        #                     "wireframe": False
        #                 }
        #             },
        #             "visualChannels": {
        #             "colorField": {
        #             "name": "Amenity",
        #             "type": "string"
        #             },
        #             "colorScale": "ordinal",
        #             "strokeColorField": None,
        #             "strokeColorScale": "quantile",
        #             "sizeField": None,
        #             "sizeScale": "linear",
        #             "heightField": None,
        #             "heightScale": "linear",
        #             "radiusField": None,
        #             "radiusScale": "linear"
        #             }
        #         }
        #     ]
        # }
    }
}
)

results_df=pd.read_csv("amenity_distances3.csv")
# print(results_df)
# Convert OSMnx graph data to a GeoDataFrame
nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(G_bbox, nodes=True, edges=True)
# Add the OSMnx nodes and edges to the map as layers
map_1.add_data(data=nodes_osmnx, name="OSMnx Nodes")
map_1.add_data(data=edges_osmnx, name="OSMnx Edges")

# Step 3: Extract unique points from 'Location' and 'Amenity Location' columns
unique_locations = results_df['Location'].apply(eval).unique()
# Create a DataFrame for unique amenity locations including 'Name'
unique_amenity_locations = results_df[['Amenity Location', 'Name', 'Amenity','Category']].apply(
    lambda row: (row['Amenity Location'], row['Name'], row['Amenity'],row['Category']), axis=1).apply(
    lambda loc: (eval(loc[0])[0], eval(loc[0])[1], loc[1], loc[2],loc[3])).unique()

# Step 4: Create GeoDataFrames from the unique points
locations_gdf = gpd.GeoDataFrame(
    geometry=[Point(lon, lat) for lat, lon in unique_locations],
    crs="EPSG:4326"
)
print(unique_amenity_locations)
amenity_locations_gdf = gpd.GeoDataFrame(
    geometry=[Point(lon, lat) for lat, lon, name, amenity,category in unique_amenity_locations],
    crs="EPSG:4326"
)
# Add the 'Name' column to amenity_locations_gdf
amenity_locations_gdf['Name'] = [name for lat, lon, name, amenity,category in unique_amenity_locations]
amenity_locations_gdf['Amenity'] = [amenity for lat, lon, name, amenity,category in unique_amenity_locations]
amenity_locations_gdf['Category'] = [category for lat, lon, name, amenity,category in unique_amenity_locations]

map_1.add_data(data=locations_gdf, name="Bike Kiosk Locations")
map_1.add_data(data=amenity_locations_gdf, name="Unique Amenity Locations")



# Group amenity locations by Amenity type and add them as separate layers
# unique_amenities = amenity_locations_gdf['Amenity'].unique()
# print(unique_amenities)
# amenity_gdfs = {}
# for amenity_type in unique_amenities:
#     subset_gdf = amenity_locations_gdf[amenity_locations_gdf['Amenity'] == amenity_type]
#     name=amenity_type+".csv"
#     subset_gdf.to_csv(name)
#     amenity_gdfs[amenity_type] = subset_gdf
    # map_1.add_data(data=subset_gdf, name=f"Amenity Type: {amenity_type}")
# print(amenity_gdfs)
# Add separate amenity layers to the map
# for amenity_type, gdf in amenity_gdfs.items():
#     map_1.add_data(data=gdf, name=f"Amenity Type: {amenity_type}")

# Create lines between locations and their corresponding amenity locations
lines = []
for _, row in results_df.iterrows():
    location = Point(eval(row['Location'])[1], eval(row['Location'])[0])
    amenity_location = Point(eval(row['Amenity Location'])[1], eval(row['Amenity Location'])[0])
    line = LineString([location, amenity_location])
    lines.append({'geometry': line, 'Distance': row['Distance']})
# print(lines)
lines_gdf = gpd.GeoDataFrame(lines, crs="EPSG:4326")
map_1.add_data(data=lines_gdf, name="Lines")

# Display the map
map_1.save_to_html(file_name='amenities_with_network_updated.html')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


[(39.1006371, -84.5109365, 'Mercantile Library', 'library', 'amenity')
 (39.1031756, -84.509386, 'Shanghai Mama’s', 'restaurant', 'amenity')
 (39.1008837, -84.5127421, "McCormick & Schmick's", 'restaurant', 'amenity')
 ... (39.12873962939653, -84.48609120306936, nan, 'playground', 'leisure')
 (39.12632218511329, -84.48243388644275, nan, 'stadium', 'leisure')
 (39.127443460706345, -84.48472166179573, 'Top Up Mart', 'convenience', 'shop')]


/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Map saved to amenities_with_network_updated.html!


In [6]:
locations_df = station_info_df  # Replace with your file path

# Create an empty DataFrame to store the results
# results = pd.DataFrame(columns=['Location', 'Category', 'Amenity', 'Name', 'Distance'])
# Create an empty list to store the data for each row
data = []
name_exception=[]


# Updated tags dictionary
tags={'amenity':True,'building':True,'historic':True,'leisure':True,'landuse':True,'tourism':True,'office':True,'shop':True}

# Loop through each location in the DataFrame
for index, row in locations_df.iterrows():
    print(index)
    lat, lon = row['lat'], row['lon']
    # Get the nearest node in the walking network to the location
    orig_node = ox.distance.nearest_nodes(G_bbox, lon, lat)
    print(orig_node)
    # Calculate the walking network distances to amenities
    for category, types in tags.items():
        try:
            amenities = ox.features.features_from_point((lat, lon), dist=500, tags={category: types})
        except Exception as e:
            print(f"Error processing category: {category} for {index}. Skipping...\n{e}")
            continue
        for idx, amenity in amenities.iterrows():
            # Access the geometry of the amenity
            amenity_geom = amenity['geometry']
            if amenity_geom.geom_type == 'Polygon':
                amenity_point = amenity_geom.centroid
                amenity_x = amenity_point.x
                amenity_y = amenity_point.y
            elif amenity_geom.geom_type == 'MultiPolygon':
                amenity_centroid = amenity_geom.centroid
                amenity_x = amenity_centroid.x
                amenity_y = amenity_centroid.y
            elif amenity_geom.geom_type == 'LineString':
                amenity_centroid = amenity_geom.centroid
                amenity_x = amenity_centroid.x
                amenity_y = amenity_centroid.y
            else:
                amenity_x = amenity_geom.x
                amenity_y = amenity_geom.y
            
            # Calculate the network distance
            dest_node = ox.distance.nearest_nodes(G_bbox, amenity_x, amenity_y)
            print(dest_node)
            try:
                distance= nx.shortest_path_length(G_bbox, orig_node, dest_node, weight='length')
                if distance <= 500:  # Check if distance exceeds 500 meters
                    
                    amenity_name = amenity.get("name", "Unknown")
                    if amenity_name == "Unknown":
                        name_exception.append(amenity)
                    # Append the data for the row to the list
                    data.append({
                        'Location': (lat, lon),
                        'Station ID':row['station_id'],
                        'Category': category,
                        'Amenity': amenity[category],
                        'Amenity Location': (amenity_y, amenity_x),
                        'Name': amenity_name,
                        'Distance': distance,
                    })
            except nx.NetworkXNoPath:
                pass  # Handle cases where no path exists

# Create a DataFrame from the list of data
results = pd.DataFrame(data)

# Print the results
print(results)

0
5471876971
6316883736
197531272
5471876965
2471276570
6821484368
1250209402
525155148
5471876971
5472286153
9765761444
1060225073
197485154
2471276565
597068901
8511324445
1250209376
1250209402
2010318658
5464268376
6833013001
6833013000
6833013000
2195187297
2774846081
2092979883
2196260099
1060108729
2085213757
2774846081
197441760
1250209402
8508265993
5472286169
2345137213
2345137217
2345137218
5471876971
2085213757
197489290
5471780396
1060108729
5471876948
9765830394
5471876860
5471876900
6821488805
197473167
2707976047
2707976066
9778403365
2707976059
2707976053
2774846081
6316880728
478217512
2046030667
7071346298
6630316238
6833083832
197522439
7610874721
5471876971
5471876837
9765864956
4394257023
6833012894
197531262
9778403356
197616190
2471274155
197616190
197499371
8508265993
197531256
197406931
9765864956
2774846077
1866317811
5471876852
197436234
477195066
6630312444
197528400
6770749508
197467520
197531262
6745824487
197531262
197621307
197489284
197479039
5471780417

In [8]:
results.to_csv('amenity_distances4.csv', index=False)
print(name_exception)

[geometry    POLYGON ((-84.4893721 39.100765, -84.489714 39...
nodes       [3462514588, 304466281, 162647755, 1260460861,...
landuse                                                retail
Name: (way, 218670724), dtype: object, geometry    POLYGON ((-84.490488 39.1009364, -84.490715 39...
nodes       [1260460807, 162893280, 162893275, 3762066704,...
landuse                                           residential
Name: (way, 372657166), dtype: object, geometry    POLYGON ((-84.4887034 39.1037407, -84.488605 3...
nodes       [256430401, 256430400, 256430399, 256430390, 2...
landuse                                            commercial
Name: (way, 467451573), dtype: object, geometry    POLYGON ((-84.4862533 39.1022414, -84.4860269 ...
nodes       [5825775055, 5825775056, 5825775057, 582577505...
landuse                                                meadow
Name: (way, 616158694), dtype: object, nodes               [358615892, 1750623236, 1750623267, 1750623142...
building                     

In [5]:
from keplergl import KeplerGl
import osmnx as ox
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point,LineString
# Create a KeplerGl map instance
map_1 = KeplerGl(height=800, 
    config={
    "version": "v1",
    "config": {
        "mapState": {
            "latitude": 39.10139,
            "longitude": -84.51164,
            "zoom": 13
        },
        # "visState": {
        #     "layers": [
        #         {
        #             "id": "unique_amenity_locations",
        #             "type": "geojson",
        #             "config": {
        #                 "dataId": "Unique Amenity Locations",
        #                 "label": "Amenity Locations",
        #                 "color": [255,203,153],
        #                 "highlightColor": [252,242,26,255],
        #                 "columns": {
        #                     "geojson": "geometry"
        #                 },
        #                 "isVisible": True,
        #                 "visConfig": {
        #                     "opacity": 0.8,
        #                     "strokeOpacity": 0.8,
        #                     "strokeColor": None,
        #                     "thickness": 0.5,
        #                     "colorRange": {
        #                         "name": "Uber Viz Qualitative 3",
        #                         "type": "qualitative",
        #                         "category": "Uber",
        #                         "colors": [
        #                             "#12939A",
        #                             "#DDB27C",
        #                             "#88572C",
        #                             "#FF991F",
        #                             "#F15C17",
        #                             "#223F9A",
        #                             "#DA70BF",
        #                             "#125C77",
        #                             "#4DC19C",
        #                             "#776E57",
        #                             "#17B8BE",
        #                             "#F6D18A",
        #                             "#B7885E",
        #                             "#FFCB99",
        #                             "#F89570"
        #                         ]
        #                     },
        #                     "strokeColorRange": {
        #                         "name": "Global Warming",
        #                         "type": "sequential",
        #                         "category": "Uber",
        #                         "colors": [
        #                         "#5A1846",
        #                         "#900C3F",
        #                         "#C70039",
        #                         "#E3611C",
        #                         "#F1920E",
        #                         "#FFC300"
        #                         ]
        #                     },
        #                     "radius": 10,
        #                     "sizeRange": [
        #                         0,
        #                         10
        #                     ],
        #                     "radiusRange": [
        #                         0,
        #                         50
        #                     ],
        #                     "heightRange": [
        #                         0,
        #                         500
        #                     ],
        #                     "elevationScale": 5,
        #                     "enableElevationZoomFactor": True,
        #                     "stroked": False,
        #                     "filled": True,
        #                     "enable3d": False,
        #                     "wireframe": False
        #                 }
        #             },
        #             "visualChannels": {
        #             "colorField": {
        #             "name": "Amenity",
        #             "type": "string"
        #             },
        #             "colorScale": "ordinal",
        #             "strokeColorField": None,
        #             "strokeColorScale": "quantile",
        #             "sizeField": None,
        #             "sizeScale": "linear",
        #             "heightField": None,
        #             "heightScale": "linear",
        #             "radiusField": None,
        #             "radiusScale": "linear"
        #             }
        #         }
        #     ]
        # }
    }
}
)

results_df=pd.read_csv("amenity_distances4.csv")
# print(results_df)
# Convert OSMnx graph data to a GeoDataFrame
nodes_osmnx, edges_osmnx = ox.graph_to_gdfs(G_bbox, nodes=True, edges=True)
# Add the OSMnx nodes and edges to the map as layers
map_1.add_data(data=nodes_osmnx, name="OSMnx Nodes")
map_1.add_data(data=edges_osmnx, name="OSMnx Edges")

# Step 3: Extract unique points from 'Location' and 'Amenity Location' columns
unique_locations = results_df['Location'].apply(eval).unique()
# Create a DataFrame for unique amenity locations including 'Name'
unique_amenity_locations = results_df[['Amenity Location', 'Name', 'Amenity','Category']].apply(
    lambda row: (row['Amenity Location'], row['Name'], row['Amenity'],row['Category']), axis=1).apply(
    lambda loc: (eval(loc[0])[0], eval(loc[0])[1], loc[1], loc[2],loc[3])).unique()

# Step 4: Create GeoDataFrames from the unique points
locations_gdf = gpd.GeoDataFrame(
    geometry=[Point(lon, lat) for lat, lon in unique_locations],
    crs="EPSG:4326"
)
print(unique_amenity_locations)
amenity_locations_gdf = gpd.GeoDataFrame(
    geometry=[Point(lon, lat) for lat, lon, name, amenity,category in unique_amenity_locations],
    crs="EPSG:4326"
)
# Add the 'Name' column to amenity_locations_gdf
amenity_locations_gdf['Name'] = [name for lat, lon, name, amenity,category in unique_amenity_locations]
amenity_locations_gdf['Amenity'] = [amenity for lat, lon, name, amenity,category in unique_amenity_locations]
amenity_locations_gdf['Category'] = [category for lat, lon, name, amenity,category in unique_amenity_locations]

map_1.add_data(data=locations_gdf, name="Bike Kiosk Locations")
map_1.add_data(data=amenity_locations_gdf, name="Unique Amenity Locations")



# Group amenity locations by Amenity type and add them as separate layers
# unique_amenities = amenity_locations_gdf['Amenity'].unique()
# print(unique_amenities)
# amenity_gdfs = {}
# for amenity_type in unique_amenities:
#     subset_gdf = amenity_locations_gdf[amenity_locations_gdf['Amenity'] == amenity_type]
#     name=amenity_type+".csv"
#     subset_gdf.to_csv(name)
#     amenity_gdfs[amenity_type] = subset_gdf
    # map_1.add_data(data=subset_gdf, name=f"Amenity Type: {amenity_type}")
# print(amenity_gdfs)
# Add separate amenity layers to the map
# for amenity_type, gdf in amenity_gdfs.items():
#     map_1.add_data(data=gdf, name=f"Amenity Type: {amenity_type}")

# Create lines between locations and their corresponding amenity locations
lines = []
for _, row in results_df.iterrows():
    location = Point(eval(row['Location'])[1], eval(row['Location'])[0])
    amenity_location = Point(eval(row['Amenity Location'])[1], eval(row['Amenity Location'])[0])
    line = LineString([location, amenity_location])
    lines.append({'geometry': line, 'Distance': row['Distance']})
# print(lines)
lines_gdf = gpd.GeoDataFrame(lines, crs="EPSG:4326")
map_1.add_data(data=lines_gdf, name="Lines")

# Display the map
map_1.save_to_html(file_name='amenities_with_network_updated.html')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


[(39.10219531873342, -84.50869437947634, 'Columbia Plaza', 'office', 'building')
 (39.10237561079389, -84.50949395088031, 'John Weld Peck Federal Building', 'office', 'building')
 (39.10134959140441, -84.50915061497938, 'PNC Center', 'office', 'building')
 ... (39.1265077986246, -84.48717724555776, nan, 'retail', 'landuse')
 (39.12595766828278, -84.48392645486344, nan, 'retail', 'landuse')
 (39.12595317447176, -84.48499090275685, nan, 'retail', 'landuse')]


/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
/Users/vaishakholla/dev/Transit/venv/lib/python3.11/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Map saved to amenities_with_network_updated.html!
